<center>

# Implementation of New Message Encryption using Elliptic Curve Cryptography Over Finite Fields
</center>

Used article: [link](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=9493519)

## Proposed method parameters

<center>

| Set-up Parameters | *n, a, b, p, G* |
|-------------------|-----------------|
| Key Generator     | *P_B, P_A, n_A, n_B, K* |
| Encryption | *E* = [*c_1c_2c_3...c_l*] |
| Decryption | *M* = [m_1m_2m_3...m_l] |
</center>